## Google's mT5

mT5 is pretrained on the mC4 corpus, covering 101 languages:

Afrikaans, Albanian, Amharic, Arabic, Armenian, Azerbaijani, Basque, Belarusian, Bengali, Bulgarian, Burmese, Catalan, Cebuano, Chichewa, Chinese, Corsican, Czech, Danish, Dutch, English, Esperanto, Estonian, Filipino, Finnish, French, Galician, Georgian, German, Greek, Gujarati, Haitian Creole, Hausa, Hawaiian, Hebrew, Hindi, Hmong, Hungarian, Icelandic, Igbo, Indonesian, Irish, Italian, Japanese, Javanese, Kannada, Kazakh, Khmer, Korean, Kurdish, Kyrgyz, Lao, Latin, Latvian, Lithuanian, Luxembourgish, Macedonian, Malagasy, Malay, Malayalam, Maltese, Maori, Marathi, Mongolian, Nepali, Norwegian, Pashto, Persian, Polish, Portuguese, Punjabi, Romanian, Russian, Samoan, Scottish Gaelic, Serbian, Shona, Sindhi, Sinhala, Slovak, Slovenian, Somali, Sotho, Spanish, Sundanese, Swahili, Swedish, Tajik, Tamil, Telugu, Thai, Turkish, Ukrainian, Urdu, Uzbek, Vietnamese, Welsh, West Frisian, Xhosa, Yiddish, Yoruba, Zulu.

In [1]:
!pip install evaluate
!pip install sacrebleu
!pip install evaluate

In [ ]:
import numpy as np
import pandas as pd
import evaluate
from datasets import load_dataset
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
from transformers import AutoTokenizer
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

checkpoint = "google/mt5-small" # to train LLM choose google/mt5-large
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## 1. Choose language to train translation model

__Availiable languages:__ ("ar", "cs"), ("ar", "de"),
    ("cs", "de"),
    ("ar", "en"),
    ("cs", "en"),
    ("de", "en"),
    ("ar", "es"),
    ("cs", "es"),
    ("de", "es"),
    ("en", "es"),
    ("ar", "fr"),
    ("cs", "fr"),
    ("de", "fr"),
    ("en", "fr"),
    ("es", "fr"),
    ("ar", "it"),
    ("cs", "it"),
    ("de", "it"),
    ("en", "it"),
    ("es", "it"),
    ("fr", "it"),
    ("ar", "ja"),
    ("cs", "ja"),
    ("de", "ja"),
    ("en", "ja"),
    ("es", "ja"),
    ("fr", "ja"),
    ("ar", "nl"),
    ("cs", "nl"),
    ("de", "nl"),
    ("en", "nl"),
    ("es", "nl"),
    ("fr", "nl"),
    ("it", "nl"),
    ("ar", "pt"),
    ("cs", "pt"),
    ("de", "pt"),
    ("en", "pt"),
    ("es", "pt"),
    ("fr", "pt"),
    ("it", "pt"),
    ("nl", "pt"),
    ("ar", "ru"),
    ("cs", "ru"),
    ("de", "ru"),
    ("en", "ru"),
    ("es", "ru"),
    ("fr", "ru"),
    ("it", "ru"),
    ("ja", "ru"),
    ("nl", "ru"),
    ("pt", "ru"),
    ("ar", "zh"),
    ("cs", "zh"),
    ("de", "zh"),
    ("en", "zh"),
    ("es", "zh"),
    ("fr", "zh"),
    ("it", "zh"),
    ("ja", "zh"),
    ("nl", "zh"),
    ("pt", "zh"),
    ("ru", "zh"),

In [ ]:
lang = "en-ru"
news_en = load_dataset("news_commentary", lang)
news_en = news_en["train"].train_test_split(test_size=0.2)

Generating train split:   0%|          | 0/190104 [00:00<?, ? examples/s]

Dataset news_commentary downloaded and prepared to /root/.cache/huggingface/datasets/news_commentary/en-ru/11.0.0/cfab724ce975dc2da51cdae45302389860badc88b74db8570d561ced6004f8b4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/160740 [00:00<?, ? examples/s]

Dataset news_commentary downloaded and prepared to /root/.cache/huggingface/datasets/news_commentary/fr-ru/11.0.0/cfab724ce975dc2da51cdae45302389860badc88b74db8570d561ced6004f8b4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/84455 [00:00<?, ? examples/s]

Dataset news_commentary downloaded and prepared to /root/.cache/huggingface/datasets/news_commentary/ar-ru/11.0.0/cfab724ce975dc2da51cdae45302389860badc88b74db8570d561ced6004f8b4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

## 2. Data preprocessing and tokenization

In [ ]:
def preprocess_function(examples):
    target_lang="ru"
    source_lang="en"
    prefix = "translate English to Russian: "
    tokenizer = AutoTokenizer.from_pretrained(checkpoint,
                                            src_lang="en",
                                            tgt_lang="ru")

    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
tokenized_news_en = news_en.map(preprocess_function, batched=True)


  0%|          | 0/153 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


  0%|          | 0/39 [00:00<?, ?ba/s]

  0%|          | 0/129 [00:00<?, ?ba/s]

  0%|          | 0/33 [00:00<?, ?ba/s]

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/17 [00:00<?, ?ba/s]

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

metric = evaluate.load("sacrebleu") # choose metric

## 3. Model building

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer =  AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
)


## 4. Model training

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_news_en["train"].select(range(10000)),
    eval_dataset=tokenized_news_en["test"].select(range(1000)),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,12.235700,3.617585,0.128100,10.429000
2,5.490700,3.197928,0.339300,14.476000


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=1250, training_loss=8.1036857421875, metrics={'train_runtime': 560.7619, 'train_samples_per_second': 35.666, 'train_steps_per_second': 2.229, 'total_flos': 1655911844413440.0, 'train_loss': 8.1036857421875, 'epoch': 2.0})

In [ ]:
trainer.save_model('/kaggle/working/my/')

## 5. Test the model

In [ ]:
from transformers import pipeline
translator = pipeline("translation", model='/working/my/')

In [ ]:
text_en = "Hello my friend"

In [ ]:
translator(text_en, src_lang='en', tgt_lang='ru')